## [Clustering with Spark](https://spark.apache.org/docs/latest/ml-clustering.html)  

This notebook describes clustering algorithms in Spark MLlib.  
The guide for clustering in the RDD-based API also has relevant information about these algorithms.

### Table of Contents

+ K-means
+ Input Columns
+ Output Columns
+ Latent Dirichlet allocation (LDA)
+ Bisecting k-means
+ Gaussian Mixture Model (GMM)
+ Input Columns
+ Output Columns
+ Power Iteration Clustering (PIC)

#### [K-means](https://github.com/apache/spark/blob/master/examples/src/main/python/mllib/k_means_example.py)  

[k-means](http://en.wikipedia.org/wiki/K-means_clustering) is one of the most commonly used clustering algorithms that clusters the data points into a predefined number of clusters. The MLlib implementation includes a parallelized variant of the k-means++ method called kmeans||.

KMeans is implemented as an Estimator and generates a KMeansModel as the base model.


<h3 id="input-columns">Input Columns</h3>

<table class="table">
  <thead>
    <tr>
      <th align="left">Param name</th>
      <th align="left">Type(s)</th>
      <th align="left">Default</th>
      <th align="left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>featuresCol</td>
      <td>Vector</td>
      <td>"features"</td>
      <td>Feature vector</td>
    </tr>
  </tbody>
</table>

<h3 id="output-columns">Output Columns</h3>

<table class="table">
  <thead>
    <tr>
      <th align="left">Param name</th>
      <th align="left">Type(s)</th>
      <th align="left">Default</th>
      <th align="left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>predictionCol</td>
      <td>Int</td>
      <td>"prediction"</td>
      <td>Predicted cluster center</td>
    </tr>
  </tbody>
</table>

In [0]:
import os
os.getcwd()

# Notes on reading files for Spark within Databricks
# https://docs.databricks.com/_static/notebooks/files-in-repos.html

Out[1]: '/Workspace/Repos/renato.rocha-souza@rbinternational.com/Embedded_Data_Scientist/Module_B/Day3'

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Loads data.
dataset = spark.read.format("libsvm").load(f"file:{os.getcwd()}/data/mllib/sample_kmeans_data.txt")

# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.9997530305375207
Cluster Centers: 
[9.1 9.1 9.1]
[0.1 0.1 0.1]


#### [Latent Dirichlet allocation (LDA)](https://github.com/apache/spark/blob/master/examples/src/main/python/ml/lda_example.py)  

LDA is implemented as an Estimator that supports both EMLDAOptimizer and OnlineLDAOptimizer, and generates a LDAModel as the base model.
Expert users may cast a LDAModel generated by EMLDAOptimizer to a DistributedLDAModel if needed.

In [0]:
from pyspark.ml.clustering import LDA

# Loads data.
dataset = spark.read.format("libsvm").load(f"file:{os.getcwd()}/data/mllib/sample_lda_libsvm_data.txt")

# Trains a LDA model.
lda = LDA(k=10, maxIter=10)
model = lda.fit(dataset)

ll = model.logLikelihood(dataset)
lp = model.logPerplexity(dataset)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

# Describe topics.
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

# Shows the result
transformed = model.transform(dataset)
transformed.show(truncate=False)

The lower bound on the log likelihood of the entire corpus: -797.5200544004555
The upper bound on perplexity: 3.0673848246171365
The topics described by their top-weighted terms:
+-----+-----------+---------------------------------------------------------------+
|topic|termIndices|termWeights                                                    |
+-----+-----------+---------------------------------------------------------------+
|0    |[1, 3, 4]  |[0.10368082673401043, 0.10246920001021176, 0.09945342991888821]|
|1    |[0, 5, 9]  |[0.1076176051626867, 0.09801051465962088, 0.09705006627909334] |
|2    |[5, 10, 9] |[0.09817190617101527, 0.0981118296756393, 0.09564406780739915] |
|3    |[5, 10, 2] |[0.10428733568856435, 0.10200642097504846, 0.09787247160826047]|
|4    |[5, 8, 2]  |[0.10610350651837704, 0.10225089640708551, 0.0969920925809682] |
|5    |[2, 1, 5]  |[0.1017814308587037, 0.09673789329662605, 0.09602700830858574] |
|6    |[3, 5, 4]  |[0.1616436533169385, 0.1391919780709568, 0.114

#### [Bisecting k-means](https://github.com/apache/spark/blob/master/examples/src/main/python/ml/bisecting_k_means_example.py)

Bisecting k-means is a kind of [hierarchical clustering](https://en.wikipedia.org/wiki/Hierarchical_clustering) using a divisive (or “top-down”) approach: all observations start in one cluster, and splits are performed recursively as one moves down the hierarchy.  

Bisecting K-means can often be much faster than regular K-means, but it will generally produce a different clustering.  

BisectingKMeans is implemented as an Estimator and generates a BisectingKMeansModel as the base model.

In [0]:
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Loads data.
dataset = spark.read.format("libsvm").load(f"file:{os.getcwd()}/data/mllib/sample_kmeans_data.txt")

# Trains a bisecting k-means model.
bkm = BisectingKMeans().setK(2).setSeed(1)
model = bkm.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
print("Cluster Centers: ")
centers = model.clusterCenters()
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.9997530305375207
Cluster Centers: 
[0.1 0.1 0.1]
[9.1 9.1 9.1]


#### [Gaussian Mixture Model (GMM)](https://github.com/apache/spark/blob/master/examples/src/main/python/ml/gaussian_mixture_example.py)

A [Gaussian Mixture Model](http://en.wikipedia.org/wiki/Mixture_model#Multivariate_Gaussian_mixture_model) represents a composite distribution whereby points are drawn from one of k Gaussian sub-distributions, each with its own probability.  
The spark.ml implementation uses the [expectation-maximization](http://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm) algorithm to induce the maximum-likelihood model given a set of samples.  

GaussianMixture is implemented as an Estimator and generates a GaussianMixtureModel as the base model.  

<h3 id="input-columns-1">Input Columns</h3>

<table class="table">
  <thead>
    <tr>
      <th align="left">Param name</th>
      <th align="left">Type(s)</th>
      <th align="left">Default</th>
      <th align="left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>featuresCol</td>
      <td>Vector</td>
      <td>"features"</td>
      <td>Feature vector</td>
    </tr>
  </tbody>
</table>

<h3 id="output-columns-1">Output Columns</h3>

<table class="table">
  <thead>
    <tr>
      <th align="left">Param name</th>
      <th align="left">Type(s)</th>
      <th align="left">Default</th>
      <th align="left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>predictionCol</td>
      <td>Int</td>
      <td>"prediction"</td>
      <td>Predicted cluster center</td>
    </tr>
    <tr>
      <td>probabilityCol</td>
      <td>Vector</td>
      <td>"probability"</td>
      <td>Probability of each cluster</td>
    </tr>
  </tbody>
</table>

In [0]:
from pyspark.ml.clustering import GaussianMixture

# loads data
dataset = spark.read.format("libsvm").load(f"file:{os.getcwd()}/data/mllib/sample_kmeans_data.txt")

gmm = GaussianMixture().setK(2).setSeed(538009335)
model = gmm.fit(dataset)

print("Gaussians shown as a DataFrame: ")
model.gaussiansDF.show(truncate=False)

Gaussians shown as a DataFrame: 
+-------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|mean                                                         |cov                                                                                                                                                                                                       |
+-------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[0.10000000000001552,0.10000000000001552,0.10000000000001552]|0.006666666666806454  0.006666666666806454  0.006666666666806454  \n0.006666666666806454  0.00666666666

#### [Power Iteration Clustering (PIC)](https://github.com/apache/spark/blob/master/examples/src/main/python/ml/power_iteration_clustering_example.py)

Power Iteration Clustering (PIC) is a scalable graph clustering algorithm developed by [Lin and Cohen](http://www.cs.cmu.edu/~frank/papers/icml2010-pic-final.pdf). From the abstract: PIC finds a very low-dimensional embedding of a dataset using truncated power iteration on a normalized pair-wise similarity matrix of the data.

spark.ml’s PowerIterationClustering implementation takes the following parameters:

+ k: the number of clusters to create
+ initMode: param for the initialization algorithm
+ maxIter: param for maximum number of iterations
+ srcCol: param for the name of the input column for source vertex IDs
+ dstCol: name of the input column for destination vertex IDs
+ weightCol: Param for weight column name

In [0]:
from pyspark.ml.clustering import PowerIterationClustering

df = spark.createDataFrame([
    (0, 1, 1.0),
    (0, 2, 1.0),
    (1, 2, 1.0),
    (3, 4, 1.0),
    (4, 0, 0.1)
], ["src", "dst", "weight"])

pic = PowerIterationClustering(k=2, maxIter=20, initMode="degree", weightCol="weight")

# Shows the cluster assignment
pic.assignClusters(df).show()

+---+-------+
| id|cluster|
+---+-------+
|  4|      0|
|  0|      1|
|  1|      1|
|  2|      1|
|  3|      0|
+---+-------+



Licensed to the Apache Software Foundation (ASF) under one or more
contributor license agreements.  See the NOTICE file distributed with
this work for additional information regarding copyright ownership.
The ASF licenses this file to You under the Apache License, Version 2.0
(the "License"); you may not use this file except in compliance with
the License.  You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.